Objective 1

In [2]:
def PofX(x):
    # Polynomial reduction in GF(2^8) based on AES's irreducible polynomial
    # P(X)=X^8+X^4+X^3+X+1 
    return (((x << 1) ^ (0x1B if x & 0x80 else 0x00)) & 0xFF)


def Multi(x, y):
    # Multiplication of two values x and y in GF(2^8)
    result = 0
    for i in range(8):
        if y & (1 << i):
            result ^= x # XOR "^"
        x = PofX(x)
    return result


# Precomputed multiplicative inverse in GF(2^8) based on AES's irreducible polynomial
inverse_table = {0: 0}  # Check if value is 0 as it has no inverse
for i in range(1, 256):
    for j in range(1, 256):
        if Multi(i, j) == 1:
            inverse_table[i] = j
            break
        
def InverseOperation(x):
    # Get the precomputed inverse from the table
    return inverse_table[x]


def AffineMapping(x):
    # Affine mapping for AES S-box
    y = 0
    for i in range(8):
        y |= ((((x >> i) ^ (x >> ((i + 4) % 8)) ^ (x >> ((i + 5) % 8)) ^ (x >> ((i + 6) % 8)) ^ (x >> ((i + 7) % 8)) ^ (0x63 >> i)) & 1) << i)
    return y


In [3]:
# Generating the S-box
print("AES S-box:")
S_box = [AffineMapping(InverseOperation(x)) for x in range(256)]
for i in range(16):
    print('  '.join(f'{S_box[j + 16 * i]:02X}' for j in range(16)))


AES S-box:
63  7C  77  7B  F2  6B  6F  C5  30  01  67  2B  FE  D7  AB  76
CA  82  C9  7D  FA  59  47  F0  AD  D4  A2  AF  9C  A4  72  C0
B7  FD  93  26  36  3F  F7  CC  34  A5  E5  F1  71  D8  31  15
04  C7  23  C3  18  96  05  9A  07  12  80  E2  EB  27  B2  75
09  83  2C  1A  1B  6E  5A  A0  52  3B  D6  B3  29  E3  2F  84
53  D1  00  ED  20  FC  B1  5B  6A  CB  BE  39  4A  4C  58  CF
D0  EF  AA  FB  43  4D  33  85  45  F9  02  7F  50  3C  9F  A8
51  A3  40  8F  92  9D  38  F5  BC  B6  DA  21  10  FF  F3  D2
CD  0C  13  EC  5F  97  44  17  C4  A7  7E  3D  64  5D  19  73
60  81  4F  DC  22  2A  90  88  46  EE  B8  14  DE  5E  0B  DB
E0  32  3A  0A  49  06  24  5C  C2  D3  AC  62  91  95  E4  79
E7  C8  37  6D  8D  D5  4E  A9  6C  56  F4  EA  65  7A  AE  08
BA  78  25  2E  1C  A6  B4  C6  E8  DD  74  1F  4B  BD  8B  8A
70  3E  B5  66  48  03  F6  0E  61  35  57  B9  86  C1  1D  9E
E1  F8  98  11  69  D9  8E  94  9B  1E  87  E9  CE  55  28  DF
8C  A1  89  0D  BF  E6  42  68  41  99  2D  

Objective 2

In [4]:
def SubBytes(state, S_box):
    for i in range(4):
        for j in range(4):
            state[i][j] = S_box[state[i][j]]  #Substituting Bytes using Pre-Created S-box
    return state



def ShiftRows(state):
    state[1] = state[1][1:] + state[1][:1]  # Shift row 1 by 1 Left
    state[2] = state[2][2:] + state[2][:2]  # Shift row 2 by 2 Left
    state[3] = state[3][3:] + state[3][:3]  # Shift row 3 by 3 Left
    return state


def MixColumns(state):
    matrix = [
        [0x02, 0x03, 0x01, 0x01],
        [0x01, 0x02, 0x03, 0x01],
        [0x01, 0x01, 0x02, 0x03],
        [0x03, 0x01, 0x01, 0x02]
    ]
    new_state = []
    for col in range(4):
        new_column = []
        for i in range(4):
            result = 0
            for j in range(4):
                result ^= Multi(matrix[i][j], state[j][col])
            new_column.append(result)
        new_state.append(new_column)
    return new_state

def AddRoundKey(state, round_key):
    for i in range(4):
        for j in range(4):
            state[i][j] ^= round_key[i][j]
    return state

def AESEncrypt(plaintext, round_keys):
    state = [[plaintext[i + 4 * j] 
        for i in range(4)] 
         for j in range(4)]
    state = AddRoundKey(state, round_keys[0])
    for round_key in round_keys[1:]:
        state = SubBytes(state)
        state = ShiftRows(state)
        state = MixColumns(state)
        state = AddRoundKey(state, round_key)
    return state


In [43]:
# Encryption of plaintext using the AES Algorithm
Plaintex = [0x00, 0x01, 0x02, 0x03, 0x04, 0x05, 0x06, 0x07, 0x08, 0x09, 0x0A, 0x0B, 0x0C, 0x0D, 0x0E, 0x0F]
RounKeys = [[[0x00, 0x00, 0x00, 0x00], [0x00, 0x00, 0x00, 0x00], [0x00, 0x01, 0x00, 0x00], [0x00, 0x00, 0x00, 0x00]]]
encrypted_state = AESEncrypt(Plaintex, RounKeys)
print("Plaintext: ",Plaintex)
print("Encryption:",encrypted_state )

Plaintext:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Encryption: [[0, 1, 2, 3], [4, 5, 6, 7], [8, 8, 10, 11], [12, 13, 14, 15]]


Objective 3

In [6]:
round_constants = [
        [0x01, 0x00, 0x00, 0x00],
        [0x02, 0x00, 0x00, 0x00],
        [0x04, 0x00, 0x00, 0x00],
        [0x08, 0x00, 0x00, 0x00],
        [0x10, 0x00, 0x00, 0x00],
        [0x20, 0x00, 0x00, 0x00],
        [0x40, 0x00, 0x00, 0x00],
        [0x80, 0x00, 0x00, 0x00],
        [0x1b, 0x00, 0x00, 0x00],
        [0x36, 0x00, 0x00, 0x00]
]
def SubWord(word):
    return [S_box[b] 
            for b in word]

def RotateWord(word):
    return word[1:] + word[:1]

def XORWords(word1, word2):
    return [b1 ^ b2 
            for b1, b2 in zip(word1, word2)]

def KeyExpansion(key):
    key_symbols = 4  #4 words in the key
    total_rounds = 11  #10 additional rounds + 1 initial
    expanded_key = []
    word = []

    for i in range(key_symbols):
        word = key[i*4:(i+1)*4]
        expanded_key.append(word)

    #Now generate the additional words that form the expanded key
    for i in range(key_symbols, key_symbols * total_rounds):
        word = expanded_key[-1]
        if i % key_symbols == 0:

            #Apply the g function every four words and XOR with Rcon
            word = XORWords(SubWord(RotateWord(word)), round_constants[i // key_symbols - 1])

        #XOR with the word `key_symbols` positions back
        word = XORWords(word, expanded_key[i - key_symbols])
        expanded_key.append(word)
    return [byte 
            for word in expanded_key 
            for byte in word]  #Flatten the list

In [7]:
# Expansion of Key using g-function
def print_expanded_key(expanded_key):
    num_rounds = 11       #Total rounds for AES-128 (including the initial round)
    bytes_per_round = 16  #16 bytes per round key
    for i in range(num_rounds):
        round_key = expanded_key[i*bytes_per_round:(i+1)*bytes_per_round]
        round_key_hex = [f"{byte:02x}" 
                         for byte in round_key]  # Convert bytes to hexadecimal format
        print(f"Round {i} key: {' '.join(round_key_hex)}")
key = [0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00]
expanded_key = KeyExpansion(key)
print_expanded_key(expanded_key)

Round 0 key: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
Round 1 key: 62 63 63 63 62 63 63 63 62 63 63 63 62 63 63 63
Round 2 key: 9b 98 98 c9 f9 fb fb aa 9b 98 98 c9 f9 fb fb aa
Round 3 key: 90 97 34 50 69 6c cf fa f2 f4 57 33 0b 0f ac 99
Round 4 key: ee 06 da 7b 87 6a 15 81 75 9e 42 b2 7e 91 ee 2b
Round 5 key: 7f 2e 2b 88 f8 44 3e 09 8d da 7c bb f3 4b 92 90
Round 6 key: ec 61 4b 85 14 25 75 8c 99 ff 09 37 6a b4 9b a7
Round 7 key: 21 75 17 87 35 50 62 0b ac af 6b 3c c6 1b f0 9b
Round 8 key: 0e f9 03 33 3b a9 61 38 97 06 0a 04 51 1d fa 9f
Round 9 key: b1 d4 d8 e2 8a 7d b9 da 1d 7b b3 de 4c 66 49 41
Round 10 key: b4 ef 5b cb 3e 92 e2 11 23 e9 51 cf 6f 8f 18 8e


Objective 4

In [39]:
#Objective 3

# Helper function to rotate a word
def rot_word(word):
    return word[1:] + word[:1]

# Rcon table for key expansion

def key_expansion(key):
    Nk = len(key) // 4  # Number of 32-bit words in the cipher key
    rounds = 10  # Number of rounds for 128-bit key size

    if Nk == 6:
        rounds = 12
    elif Nk == 8:
        rounds = 14

    # Initialize key schedule with the initial key
    key_schedule = [key[i:i+4] 
                    for i in range(0, len(key), 4)]

    # Generate remaining words of the key schedule
    rcon_i= 0  # Initialize round_constants index
    for i in range(Nk, 4 * (rounds + 1)):
        temp = key_schedule[i - 1][:]  # Make a copy of the last word

        if i % Nk == 0:

            # Perform subword transformation    
            temp = [S_box[byte] 
                    for byte in rot_word(temp)]

            # Perform round_constants operation
            temp = [temp[i] ^ round_constants[rcon_i][i] 
                    for i in range(4)]
            rcon_i += 1  # Increment round_constants index

        elif Nk > 6 and i % Nk == 4:
            # For 256-bit key size, perform subword transformation without Rcon
            temp = [S_box[byte] 
                    for byte in temp]

        # XOR with the word previous key position 
        temp = [temp[j] ^ key_schedule[i - Nk][j] 
                for j in range(4)]
        key_schedule.append(temp)

    return key_schedule




In [41]:
# Objective 2
Nb = 4  # Block size in 32-bit words
Nk = 4  # Key size in 32-bit words
Nr = 10 # Number of rounds
def sub_bytes(state, S_box):
    new_state = []
    for row in state:
        new_row = [S_box[byte] 
                   for byte in row]
        new_state.append(new_row)
    return new_state

def shift_rows(state):
    new_state = []
    for i, row in enumerate(state):
        new_row = row[i:] + row[:i]
        new_state.append(new_row)
    return new_state

def mix_columns(state):
    # Define the fixed matrix for MixColumns
    matrix = [
        [0x02, 0x03, 0x01, 0x01],
        [0x01, 0x02, 0x03, 0x01],
        [0x01, 0x01, 0x02, 0x03],
        [0x03, 0x01, 0x01, 0x02]
    ]
    new_state = []
    for col in range(4):
        new_column = []
        for i in range(4):
            result = 0
            for j in range(4):
                result ^= Multi(matrix[i][j], state[j][col])
            new_column.append(result)
        new_state.append(new_column)
    return new_state

def add_round_key(state, round_key):
    new_state = []
    for i in range(4):
        new_row = [state[j][i] ^ round_key[j][i]
                    for j in range(4)]
        new_state.append(new_row)
    return new_state


def AES_encrypt(plaintext, key):
    # Generate key schedule
    key_schedule = key_expansion(key)

    # Initialize state with plaintext
    state = [plaintext[i:i + 4] 
             for i in range(0, len(plaintext), 4)]

    # Add initial round key
    state = add_round_key(state, key_schedule[:4])

    # Perform 10 rounds
    for round_num in range(1, 10):
        state = sub_bytes(state, S_box)
        state = shift_rows(state)
        state = mix_columns(state)
        round_key = key_schedule[4 * round_num:4 * (round_num+1)]
        state = add_round_key(state, round_key)

    # Final round
    state = sub_bytes(state, S_box)
    state = shift_rows(state)
    state = add_round_key(state, key_schedule[10*4:])

    # Flatten the state matrix to obtain ciphertext
    ciphertext = [byte 
                  for row in state 
                    for byte in row]
    return ciphertext


In [42]:
plaintext = [0x00, 0x01, 0x02, 0x03, 0x04, 0x05, 0x06, 0x07, 0x08, 0x09, 0x0A, 0x0B, 0x0C, 0x0D, 0x0E, 0x0F]
key = [0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00]


plaintext_hex = [f"{byte:02x}" 
                         for byte in plaintext]
key_hex= [f"{byte:02x}" 
                         for byte in key]
print ("The Plaintext is: ",plaintext_hex)
print ("The Input key is:",key_hex)


ciphertext = AES_encrypt(plaintext, key)
ciphertext_hex=[f"{byte:02x}" 
                         for byte in ciphertext]
print("The Ciphertext is: ",ciphertext_hex)


# Checking the pre computed ciphertext matches the AES_encrypt one
expected_ciphertext = [122, 202, 15, 21, 188, 214, 236, 124, 159, 151, 70, 102, 22, 230, 162, 130]
expected_ciphertext_hex=[f"{byte:02x}" 
                         for byte in expected_ciphertext]
print("The expected Ciphertext is:",expected_ciphertext_hex)


if ciphertext == expected_ciphertext:
    print("Encryption is correct.")
else:
    print("Encryption is incorrect.")


The Plaintext is:  ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '0a', '0b', '0c', '0d', '0e', '0f']
The Input key is: ['00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00']
The Ciphertext is:  ['7a', '1b', '77', '7d', '6d', 'd6', 'e7', 'e2', 'e7', '9c', '46', 'b1', 'b2', '78', '75', '82']
The expected Ciphertext is: ['7a', 'ca', '0f', '15', 'bc', 'd6', 'ec', '7c', '9f', '97', '46', '66', '16', 'e6', 'a2', '82']
Encryption is incorrect.


Bonus Decryption

In [ ]:
def InverseS_box(s_box):
    inv_S_box = [0] * 256
    for i in range(256):
        inv_S_box[s_box[i]] = i
    return inv_S_box

inv_S_box = InverseS_box(S_box)

# Display the inverse S-Box
print("Inverse S-Box:")
for i in range(16):
    print("  ".join(f"{inv_S_box[i * 16 + j]:02X}" 
                    for j in range(16)))

Inverse S-Box:
52  09  6A  D5  30  36  A5  38  BF  40  A3  9E  81  F3  D7  FB
7C  E3  39  82  9B  2F  FF  87  34  8E  43  44  C4  DE  E9  CB
54  7B  94  32  A6  C2  23  3D  EE  4C  95  0B  42  FA  C3  4E
08  2E  A1  66  28  D9  24  B2  76  5B  A2  49  6D  8B  D1  25
72  F8  F6  64  86  68  98  16  D4  A4  5C  CC  5D  65  B6  92
6C  70  48  50  FD  ED  B9  DA  5E  15  46  57  A7  8D  9D  84
90  D8  AB  00  8C  BC  D3  0A  F7  E4  58  05  B8  B3  45  06
D0  2C  1E  8F  CA  3F  0F  02  C1  AF  BD  03  01  13  8A  6B
3A  91  11  41  4F  67  DC  EA  97  F2  CF  CE  F0  B4  E6  73
96  AC  74  22  E7  AD  35  85  E2  F9  37  E8  1C  75  DF  6E
47  F1  1A  71  1D  29  C5  89  6F  B7  62  0E  AA  18  BE  1B
FC  56  3E  4B  C6  D2  79  20  9A  DB  C0  FE  78  CD  5A  F4
1F  DD  A8  33  88  07  C7  31  B1  12  10  59  27  80  EC  5F
60  51  7F  A9  19  B5  4A  0D  2D  E5  7A  9F  93  C9  9C  EF
A0  E0  3B  4D  AE  2A  F5  B0  C8  EB  BB  3C  83  53  99  61
17  2B  04  7E  BA  77  D6  26  E1  69  